In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.5.7`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._

val conf = new SparkConf().setAppName("RDD").setMaster("local[*]").set("spark.log.level", "WARN")
val sc = new SparkContext(conf)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/24 20:55:04 INFO SparkContext: Running Spark version 3.5.7
26/01/24 20:55:04 INFO SparkContext: OS info Linux, 6.8.0-90-generic, amd64
26/01/24 20:55:04 INFO SparkContext: Java version 1.8.0_442
26/01/24 20:55:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/24 20:55:04 INFO ResourceUtils: ==============================================================
26/01/24 20:55:04 INFO ResourceUtils: No custom resources configured for spark.driver.
26/01/24 20:55:04 INFO ResourceUtils: ==============================================================
26/01/24 20:55:04 INFO SparkContext: Submitted application: Hello Spark
26/01/24 20:55:04 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), t

spark.version == 3.5.7


import $ivy.$
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import java.sql.Timestamp
import java.time._
spark: SparkSession = org.apache.spark.sql.SparkSession@6ad66e46
import spark.implicits._

In [ ]:
val tripRaw = sc.textFile("data/tripdata.csv")
val taxiRaw = sc.textFile("data/taxi_zone_lookup.csv")

val tripHeader = tripRaw.first()
val taxiHeader = taxiRaw.first()

val trip = tripRaw.filter(_ != tripHeader)
val taxi = taxiRaw.filter(_ != taxiHeader)

In [ ]:

val tripParsed = trip.map(line => line.split(","))

In [ ]:

val tripParsed = trip.map { line =>
  val arr = line.split(",")
  try {
    val hour = arr(1).substring(11, 13).toInt
    val puLocationId = arr(7).toInt
    (puLocationId, hour)
  } catch {
    case _: Throwable => (-1, -1)
  }
}.filter { case (pu, h) => pu != -1 && h != -1 }

tripParsed.count()

In [ ]:
val taxiParsed = taxi.map { line =>
  val arr = line.split(",")
  val locationId = arr(0).toInt
  val borough = arr(1)
  (locationId, borough)
}

taxiParsed.count()

In [ ]:
val joined = tripParsed.join(taxiParsed)

val ordersByBoroughHour = joined
  .map { case (_, (hour, borough)) => ((borough, hour), 1) }
  .reduceByKey(_ + _)

val result = ordersByBoroughHour.map {
  case ((borough, hour), count) => s"$borough,$hour,$count"
}

result.saveAsTextFile("data/orders_by_borough_hour")